# Data preprocessing

## Importer les dépendances

In [28]:
import pandas as pd
import numpy as np

## Importer le dataset

In [29]:
dataset = pd.read_csv('data/Data.csv')
#print(dataset.head())

iloc prend deux parametres : nombre de lignes et nombre de colonnes ":" signifie que l'on prend toutes les lignes ou colonnes du dataset en faisant ":-1" on part du principe que la derniere colonne contient la colonne de variables dependantes que l'on mettra dans le vecteur y

In [30]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values # on recupere juste la derniere colonne

## Gérer les données manquantes
Pour cela, on utilise la class Imputer du module preprocessing

In [31]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)

On lie l'imputer a notre variable en precisant les lignes puis colonnes que l'on souhaite. Pour rappel, dans un range, la borne superieure est toujours exclue en python donc 1:3 correspond aux colonnes dont l'indice est 1 et 2

In [32]:
imputer.fit(X[:, 1:3])
# on transforme la variable conformement a la strategie definie 
X[:, 1:3] = imputer.transform(X[:, 1:3])
print("Independent dataset X:")
print(X)
print("")

Independent dataset X:
[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]



## Gérer les variables catégoriques (ie non numériques continues)

Ici dans notre dataset, les pays (3 categories) et purchase (2 categories). Le but est d'encoder les variables catégoriques (en texte par exemple) en variable numerique. 

Variable nominale (= non ordonnee) vs. ordinale (= ordonnee).

La class LabelEncoder permet de transformer du texte en valeur numérique (1, 2, 3, ...) et la class OneHotEncoder permet de spliter en plusieurs colonnes une colonne contenant des valeurs numeriques (= pas de texte)

In [33]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder() # pour la variable country
# fit_transform fait d'abord fit puis transform (en evitant des calculs en double)
X[:, 0] = labelencoder_X.fit_transform(X[:, 0]) # country est la 1ere colonne

Méthode d'encodage pour une variable catégorique nominale = encode par dummy-variable (obligatoire pour les variables independantes = entrees).

L'encodage par dummy-variable consiste a decouper en N colonnes une variable categorique à N categories (ie. valeurs distinctes) puis de mettre 1 ou 0 dans chacune de ces colonnes (par exemple la valeur France de la colonne country est remplacée par un 1 dans la colonne France et 0 dans les colonnes Germany et Spain). Si on avait simplement remplacé les pays par un chiffre (0 pour la France, 1 pour Germany et 2 pour Spain), on aurait recréé une relation d'ordre entre les pays puisque 1 < 2 etc.

In [34]:
onehotencoder = OneHotEncoder(categorical_features = [0], dtype=int) # categorical_features correspond aux indices des colonnes a transformer
np.set_printoptions(suppress=True)
X = onehotencoder.fit_transform(X).toarray()
print("Independent dataset X apres gestion des variables categoriques:")
print(X)
print("")

Independent dataset X apres gestion des variables categoriques:
[[     1.              0.              0.             44.          72000.        ]
 [     0.              0.              1.             27.          48000.        ]
 [     0.              1.              0.             30.          54000.        ]
 [     0.              0.              1.             38.          61000.        ]
 [     0.              1.              0.             40.          63777.77777778]
 [     1.              0.              0.             35.          58000.        ]
 [     0.              0.              1.             38.77777778  52000.        ]
 [     1.              0.              0.             48.          79000.        ]
 [     0.              1.              0.             50.          83000.        ]
 [     1.              0.              0.             37.          67000.        ]]



## Transformation de la variable 'purchase' textuelle en valeurs numériques

In [35]:
labelencoder_y = LabelEncoder() 
y = labelencoder_y.fit_transform(y)

## Diviser le dataset entre le Training set et le Test set

Le training set est la base d'apprentissage pour découvrir les corrélations entre les variables independantes (entrees) et les variables dependantes (sorties) afin d'eviter le sur-apprentissage (apprentissage par coeur), on  conserve une partie des donnees pour tester le modele d'apprentissage avec les valeurs attendues.

Les observations du test set n'auront pas contribué à l'apprentissage et on dimensionne a 20% des observations totales, le nombre d'observations à mettre de coté pour le test set.

Si la precision (taux d'erreur) du test set est significativement plus élevée que le train set, cela signifie qu'il y a eu sur-apprentissage.

In [36]:
from sklearn.model_selection import train_test_split
# on renseigne le random_state pour est sur d'avoir les mêmes resultats a 
# chaque execution : utile uniquement dans un but pedagogique ou de debugging
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature scaling

On met à la même echelle toutes les variables pour éviter qu'une variable d'entrée n'écrase toutes les autres (par exemple, dans notre dataset la colonne Age n'a pas du tout la même échelle que la colonne Salaire).

On utilise ici la méthode de standardisation (qui consiste à soustraire aux valeurs de x la moyenne des valeurs de x le tout divisé par l'écart-type de x).

Une autre methode est la methode de normalisation (qui consiste à soustraire aux valeurs de x la valeur minimum de x le tout divisé par la soustraction de la valeur maximum de x par la valeur minimum de x).

In [37]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#print(X_train)
X_train = sc.fit_transform(X_train)

Grâce la fonction train_test_split, le training set et le test set ont une moyenne et un écart-type similaire et une distribution similaire des valeurs.

On n'a donc pas besoin de fitter sc à X_test puisqu'il est déjà fitté à X_train.

In [38]:
X_test = sc.transform(X_test)
#print(X_train)

On n'applique par le feature scaling sur y (sorties) puisque dans ce dataset, les valeurs ont déjà été encodées (avec LabelEncoder) en 0 ou 1 ce qui est dans une echelle similaire aux variables de X (entrees).

En fonction du dataset, il peut être nécessaire d'appliquer le feature scaling aux sorties (typiquement si y est un vecteur de valeurs numériques avec une échelle très différentes des valeurs de X)